# Ejemplo py_pol

Ejemplo inicial para comenzar a utilizar el módulo py_pol en frmalismo de Jones.

In [ ]:
from py_pol.jones_vector import Jones_vector
from py_pol.jones_matrix import Jones_matrix, degrees
import numpy as np
import matplotlib.pyplot as plt

## Primeros pasos

En primer lugar vamos a ver algunos ejemplos de cómo realizar las operaciones más básicas.

### Crear objetos

In [ ]:
# Crear un vector de Jones
E = Jones_vector(name="Light")
print(E)

In [ ]:
# Crear una matriz de Jones
J = Jones_matrix(name="Optical element")
print(J)

### Definir qué es cada objeto

In [ ]:
# Crear un haz linealmente olarizado a 30º y de intensidad 2
E = Jones_vector("Laser")
E.linear_light(azimuth=30 * degrees, intensity=2)
print(E)

In [ ]:
# Crear un polarizador ideal a 10º
J = Jones_matrix("Polarizer")
J.diattenuator_perfect(azimuth=10 * degrees)
print(J)

In [ ]:
# Crear polarizadores ideales con angulos entre 0 y 90º
J = Jones_matrix("Polarizer")
angles = np.linspace(0, 90 * degrees, 5)
J.diattenuator_perfect(azimuth=angles)
print(J)

### Manipular objetos

In [ ]:
# Crear un haz linealmente polarizado a 0º y despues rotarlo 45º
E = Jones_vector("Laser")
E.linear_light(azimuth=0 * degrees)
print(E)

E.rotate(angle=45 * degrees)
print(E)

In [ ]:
# Crear un polarizador ideal a 0º y despues rotarlo 45º
J = Jones_matrix("Polarizer")
J.diattenuator_perfect(azimuth=0 * degrees)
print(J)

J.rotate(angle=45 * degrees)
print(J)

### Operaciones entre objetos

In [ ]:
# Calcular el haz resultante de un haz linealmente polarizado a 0º tras atravesar un polarizador ideal a 45º
E = Jones_vector("Laser")
E.linear_light(azimuth=0 * degrees)
print(E)

J = Jones_matrix("Polarizer")
J.diattenuator_perfect(azimuth=45 * degrees)
print(J)

E2 = J * E
print(E2)

In [ ]:
# Calcular la matriz de Jones de un sistema compuesto por una lamina de cuarto de onda a 45º seguida de un polarizador lineal a 0º
Jr = Jones_matrix("Retarder")
Jr.quarter_waveplate(azimuth=45 * degrees)
print(Jr)

Jp = Jones_matrix("Polarizer")
Jp.diattenuator_perfect(azimuth=0 * degrees)
print(Jp)

# Note, the element that is furthest from the system is placed on the right.
Jtotal = Jr * Jp
print(Jtotal)

In [ ]:
# Calcular la matriz de Jones de un sistema compuesto por una lamina de cuarto de onda a 45º seguida de un polarizador lineal con un angulo entre 0 y 90º
E = Jones_vector("Laser")
E.linear_light(azimuth=0 * degrees)
print(E)

J = Jones_matrix("Polarizer")
angles = np.linspace(0, 90 * degrees, 5)
J.diattenuator_perfect(azimuth=angles)
print(J)

E2 = J * E
print(E2)

### Extraer parámetros de los objetos

In [ ]:
# Calcular el azimut de un haz eliptico
angles = np.linspace(0, 180 * degrees, 19)
E = Jones_vector("laser")
E.general_azimuth_ellipticity(intensity=2, azimuth=angles, ellipticity=20 * degrees)

# Extra parameters can be obtained from the
E.parameters.get_all(draw=True)

In [ ]:
# calcular la retardancia de 100 elementos opticos completamente aleatorios
Jelements = np.random.rand(2, 2, 10, 10) + 1j * np.random.rand(2, 2, 10, 10)
J = Jones_matrix("Random element")
J.from_matrix(Jelements)
R = J.parameters.retardance(draw=True)

### Parametros utiles de los objetos py_pol

In [ ]:
E = Jones_vector("Laser")
E.linear_light(azimuth=45 * degrees, intensity=np.random.rand(3) * 5)

print("Array: ", E.M)
print("Name: ", E.name)
print("Size: ", E.size)
print("Shape: ", E.shape)
print("Number of dimensions: ", E.ndim)
print("Type: ", E.type, "\n")

I = E.parameters.intensity(verbose=True)

cond = E.checks.is_circular(verbose=True)

## Ejemplo 1: Ley de Malus

Vamos a ver la Ley de Malus (https://en.wikipedia.org/wiki/Polarizer) para el caso de polarizadores ideales y polarizadores reales.

En el caso de los polarizadores ideales, la transmisión será 1 cuando el eje de transmisión de los polarizadores estén alineados y 0 cuando estén cruzados 

$I(\theta) = I_{0} * cos(\theta_1 - \theta_0)^2$,

siendo $I_0$ la intensidad del haz circularmente polarizado incidente y $\theta_i$ el ángulo de los polarizadores.

En el caso de polarizadores reales es un poquito más complicado. Por un lado tenemos que la transmisión máxima de los polarizadores ($p_1$ en campo eléctrico o $T_{max} = p_1^2$ en intensidad) es menor que 1. Por otro lado la transmisión mínima de los polarizadores ($p_2$ en campo eléctrico o $T_{min} = p_2^2$ en intensidad) es mayor que 0. En este caso, la ecuación de la Ley de Malus se complica:

$I(\theta) = \frac{I_{0}}{2} * [T^2_{min} + T^2_{max} + (2 * T_{max} * T_{min} - T^2_{max} - T^2_{min}) * cos(\theta_1 - \theta_0)^2]$.

### Polarizadores ideales

In [ ]:
angles = np.linspace(0, 360 * degrees, 361)
J0 = Jones_matrix("Polarizer 0")
J0.diattenuator_perfect(azimuth=0)

J1 = Jones_matrix("Polarizer 1")
J1.diattenuator_perfect(azimuth=angles)

E0 = Jones_vector("Laser")
E0.circular_light(kind="l", intensity=2)

Efin = J1 * J0 * E0
I = Efin.parameters.intensity(draw=True)

### Polarizadores reales

In [ ]:
angles = np.linspace(0, 360 * degrees, 361)
J0 = Jones_matrix("Polarizer 0")
J0.diattenuator_linear(azimuth=0, Tmax=0.9, Tmin=0.1)

J1 = Jones_matrix("Polarizer 1")
J1.diattenuator_linear(azimuth=angles, Tmax=0.9, Tmin=0.1)

E0 = Jones_vector("Laser")
E0.circular_light(kind="l", intensity=2)

Efin = J1 * J0 * E0
I = Efin.parameters.intensity(draw=True)

## Ejemplo 2: Efecto Zenon clásico

El efecto Zenon clásico estudia la transmisión de un sistema de 2 o más polarizadores donde sus ángulos están equiespaciados y el primero y el último estan cruzados.

### Polarizadores ideales

In [ ]:
E0 = Jones_vector("Laser")
E0.circular_light(kind="l", intensity=2)

arr = [*range(2, 102)]
I = np.zeros(100)
for i, element in enumerate(arr):
    angles = np.linspace(0, 90 * degrees, element)
    J1 = Jones_matrix("Polarizer 1")
    J1.diattenuator_perfect(
        azimuth=angles,
    )

    # Usually the system creates one operation per polarized. We do .prod() to ask the sobtware to create the product of all the elements
    Efin = J1.prod() * E0
    I[i] = Efin.parameters.intensity()

plt.plot(arr, I)
plt.xlabel("Number of polarizers")
plt.ylabel("Final intensity")

### Polarizadores reales

In [ ]:
E0 = Jones_vector("Laser")
E0.circular_light(kind="l", intensity=2)

Narray = np.linspace(2, 101, 100)
I2 = np.zeros(100)
for ind, N in enumerate(Narray):
    angles = np.linspace(0, 90 * degrees, int(N))
    J1 = Jones_matrix("Polarizer 1")
    J1.diattenuator_linear(azimuth=angles, Tmax=0.99, Tmin=1e-4)

    Efin = J1.prod() * E0
    I2[ind] = Efin.parameters.intensity()

plt.plot(Narray, I2)
plt.plot(Narray, I, "b--")
plt.xlabel("Number of polarizers")
plt.ylabel("Final intensity")